In [1]:
import numpy as np

In [2]:
%run 'yolov3_tf2.ipynb'

In YOLOv3, there are 2 convolutional layer types i.e. with and without batch normalization. So, the weights are applied differently for different types of convolutional layers.

__The weights are stored in following format:__

`Convolution with Batch Normalization:` 

            gamma
            beta
            mean
            variance
            conv weights

`Convolution without Batch Normalization:` 
            
            conv biases
            conv weights
            
When we re-write these weights to TensorFlow's format, we need to switch the position if `beta` and `gamma` for convolution with batch normalization, so that they are ordered like: `beta`, `gamma`, `means`, `variance` and `conv weights`. However, the weights order for convolution without batch normalization remains the same.

In [3]:
def load_weights(model, cfgfile, weightfile):
    # Open the weights file
    fp = open(weightfile, 'rb')
    
    # Skip first 5 values as they are header values
    np.fromfile(fp, dtype= np.int32, count=5)
    
    blocks = parse_cfg(cfgfile)

    for i, block in enumerate(blocks[1:]):
        if(block['type'] == 'convolutional'):
            conv_layer = model.get_layer('conv_' + str(i))
            print('layer:', i+1, conv_layer)
            
            filters = conv_layer.filters
            k_size = conv_layer.kernel_size[0]
            in_dim = conv_layer.input_shape[-1]
            
            # For batch normalization, get the relevant values and re-arrange them to 
            # Tensorflow's order
            if('batch_normalize' in block):
                norm_layer = model.get_layer('bnorm_'+str(i))
                print('layer:',i+1,norm_layer)
                size = np.prod(norm_layer.get_weights()[0].shape)
                
                bn_weights = np.fromfile(fp, dtype = np.float32, count = 4 *filters)
                # tf [gamma, beta, mean, variance]
                bn_weights = bn_weights.reshape((4, filters))[[1,0,2,3]]
            # If not batch normalization, then take the bias values
            else:
                conv_bias = np.fromfile(fp, dtype= np.float32, count = filters)
            
            # darknet shape (out_dim, in_dim, height, width)
            conv_shape = (filters, in_dim, k_size, k_size)
            conv_weights = np.fromfile(
                fp, dtype=np.float32, count=np.product(conv_shape))
            # tf shape (height, width, in_dim, out_dim)
            conv_weights = conv_weights.reshape(
                conv_shape).transpose([2, 3, 1, 0])
            
            if 'batch_normalize' in block:
                norm_layer.set_weights(bn_weights)
                conv_layer.set_weights([conv_weights])
            else:
                conv_layer.set_weights([conv_weights, conv_bias])
    
    fp.close()

In [5]:
weightfile = "weights/yolov3.weights"
cfgfile = "cfg/yolov3.cfg"

model_size = (416, 416,3)
num_classes=9

model = YOLOv3Net(cfgfile,model_size, num_classes)
load_weights(model,cfgfile,weightfile)

try:
    model.save_weights('weights/yolov3_weights.tf')
    print('\nThe file \'yolov3_weights.tf\' has been saved successfully.')
except IOError:
    print("Couldn't write the file \'yolov3_weights.tf\'.")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 416, 416, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 416, 416, 32) 864         tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 416, 416, 32) 128         conv_0[0][0]                     
______________________________________________________________________________________________

layer: 21 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x63b2f2908>
layer: 21 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63b366f98>
layer: 23 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x63b3d8390>
layer: 23 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63b404780>
layer: 24 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x63b40a198>
layer: 24 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63b488908>
layer: 26 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x63b4dbe10>
layer: 26 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63b50feb8>
layer: 27 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x63b51f7b8>
layer: 27 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x63b595eb8>
layer: 29 <tensorflow.python.keras.layers.convolutional.Conv2D ob

layer: 92 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x65ccee860>
layer: 92 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x65cd64fd0>
layer: 93 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x65cdd53c8>
layer: 93 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x65ce01390>
layer: 94 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x65ce5b5c0>
layer: 97 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x65cec4da0>
layer: 97 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x65ced38d0>
layer: 100 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x639e5ef60>
layer: 100 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x65cf6fac8>
layer: 101 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x65cf7b208>
layer: 101 <tensorflow.python.keras.layers.normalization_v2.BatchNormalizatio